# Блочное шифрование - SP-сеть

## Init

### Imports

In [ ]:
from random import shuffle

### Implementation

#### Def-s

In [ ]:
def charToCharBits(c: str, bytesAmount : int = 2) -> str:
    if len(c) > 1: 
        raise OverflowError(c)
    return f"{ord(c):b}".rjust(8 * bytesAmount, "0")


def strToStrBits(s: str, bytesAmount : int = 2, separator: str = "") -> str:
    return separator.join(charToCharBits(c, bytesAmount) for c in s)


def charBitsToChar(c: str, bytesAmount: int = 2) -> str:
    if len(c) > bytesAmount * 8: 
        raise OverflowError(c)
    return chr(int(c, 2))


def strBitsToStr(s: str, bytesAmount: int = 2) -> str:
    res = ""
    step = bytesAmount * 8
    for i in range(0, int(len(s) / step)):
        res += charBitsToChar(s[i * step : (i + 1) * step])
    return res


#### Class

In [ ]:
class SPNetwork:
    """Custom SP-network"""
    def __init__(self, cryptoSequence: list, dataUnitSizeInBytes: int = 2, SBlocksAmount: int = 4, permutationAmount: int = 64):
        """Initialize new custom SP network

        Args:
            cryptoSequence (list): Sequence of application of the S- (False) or P-block (True): 
                [True, False, True, False] - that mean that word firsly go 
                into P-block, after - S-block a twice; and then into P-block again; 
                Modified key value will be returned as encoded key after going throgh blocks.
            dataUnitSizeInBytes (int, optional): Max key size in bytes.
            SBlocksAmount (int, optional): S-Block parts amount.
            permutationAmount (int, optional): Permutation sequences amount.
        """
        self.__dataUnitSizeInBytes__ = dataUnitSizeInBytes
        self.__dataUnitSizeInBits__ = dataUnitSizeInBytes * 8
        self.__SBlocksAmount__ = SBlocksAmount
        self.__cryptoSequence__ = cryptoSequence
        self.__dataSizeInBitsPerSBlockPart__ = int(dataUnitSizeInBytes * 8 / SBlocksAmount)
        self.__permutationsAmount__ = permutationAmount
        self.__permutations__ = [[] for i in range(permutationAmount)]
        self.__SBlockDictionaty__ = {}
        self.__SBlockReversedDictionaty__ = {}
        self.__InitPermutations__()
        self.__InitSubstritutions__()
        

    def __InitSubstritutions__(self):
        rc = range(2 ** self.__dataSizeInBitsPerSBlockPart__)
        first = list(rc)
        second = list(rc)
        shuffle(first)
        shuffle(second)
        for i in rc:
            self.__SBlockDictionaty__[first[i]] = second[i]
            self.__SBlockReversedDictionaty__[second[i]] = first[i]


    def __InitPermutations__(self, repeatsAmount: int = 64):
        for permutationNumber in range(self.__permutationsAmount__):
            self.__permutations__[permutationNumber] = list(range(self.__dataUnitSizeInBits__))
            for repeatNumber in range(repeatsAmount):
                shuffle(self.__permutations__[permutationNumber])


    def __PBlock__(self, wordCharBitsStr: str, permutationNumber: int, isReversed: bool = False) -> str:
        permutation = self.__permutations__[permutationNumber]
        buffer = list(wordCharBitsStr)
        counter = 0
        for index in permutation:
            if isReversed:
                buffer[index] = wordCharBitsStr[counter]
            else:
                buffer[counter] = wordCharBitsStr[index]
            counter += 1
        return ''.join(c for c in buffer)


    def __SBlock__(self, wordCharPartBitsStr: str, isReversed: bool = False) -> str:
        key = int(wordCharPartBitsStr, 2)
        value = self.__SBlockDictionaty__[key]
        res = self.__SBlockReversedDictionaty__[key] if isReversed else value
        return f'{res:b}'.rjust(self.__dataSizeInBitsPerSBlockPart__, '0')


    def Crypt(self, word: str, permutationNumber: int, isReversed: bool = False) -> str:
        if (permutationNumber < 0 or permutationNumber >= self.__permutationsAmount__):
            raise IndexError(permutationNumber)
        
        cryptoSequence = self.__cryptoSequence__

        if isReversed:
            cryptoSequence.reverse()

        resChars = strToStrBits(word, self.__dataUnitSizeInBytes__, ' ').split(' ')
        charsAmount = len(word)
        for isPBlock in cryptoSequence:
            if isPBlock:
                for charIndex in range(charsAmount):
                    resChars[charIndex] = self.__PBlock__(resChars[charIndex], permutationNumber, isReversed)
            else:
                for charIndex in range(charsAmount):
                    buffer = ""
                    for charPartIndex in range(self.__SBlocksAmount__):
                        buffer += self.__SBlock__(
                            resChars[charIndex][charPartIndex * self.__dataSizeInBitsPerSBlockPart__
                                                : (charPartIndex + 1) * self.__dataSizeInBitsPerSBlockPart__],
                            isReversed)
                    resChars[charIndex] = buffer

        return strBitsToStr(''.join(char for char in resChars), self.__dataUnitSizeInBytes__)

## Tests

In [17]:
spn = SPNetwork([False], permutationAmount=2)

In [18]:
permutNum = 1
word = 'Test'
encoded = spn.Crypt(word, permutNum)
decoded = spn.Crypt(encoded, permutNum, True)
print(f'Encoded: {encoded}')
print(f'Decoded: {decoded}')
print(decoded == word)

Encoded: 衽蠧蠔蠝
Decoded: Test
True
